In [142]:
from bertopic import BERTopic
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import random
import numpy as np
from bertopic import BERTopic
from hdbscan import HDBSCAN

In [143]:
df = pd.read_csv("/Users/vishalsehgal/Desktop/NLP/Assignment-1/Dataset/assignment-2-data.csv")

nvidia_articles = df[df['clean_content'].str.contains("nvidia", case=False, na=False)]['clean_content'].tolist()

# 1st Iteration

In [144]:
import numpy as np
import random
import torch
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from hdbscan import HDBSCAN
import os

# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

# UMAP configuration for reproducibility
umap_model = UMAP(
    random_state=SEED,
    n_neighbors=15,
    min_dist=0.1,
    metric="cosine",
    n_jobs=1
)

# HDBSCAN configuration for reproducibility
hdbscan_model = HDBSCAN(
    min_cluster_size=10,
    min_samples=10,
    prediction_data=True,
    approx_min_span_tree=False,
    core_dist_n_jobs=1
)

# Set up CountVectorizer
vectorizer_model = CountVectorizer(lowercase=True)

# Initialize BERTopic without fixed nr_topics to allow natural topic count
bert_topic = BERTopic(
    vectorizer_model=vectorizer_model,
    min_topic_size=10,
    calculate_probabilities=True,
    embedding_model="all-MiniLM-L6-v2",
    umap_model=umap_model,
    hdbscan_model=hdbscan_model
)

# Re-seed before fit_transform
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)

# Fit and transform data
topic_assigned_to_doc, probabilities = bert_topic.fit_transform(nvidia_articles)

# Retrieve initial topic count
initial_topic_count = bert_topic.get_topic_freq().shape[0] - 1  # Exclude -1 (outliers)

# Reduce topics only if significantly above target
target_nr_topics = 60
if initial_topic_count > target_nr_topics:
    bert_topic.reduce_topics(nvidia_articles, nr_topics=target_nr_topics)

# Retrieve and display topics
topic_matrix = bert_topic.get_topic_info()
for topic_id in topic_matrix['Topic']:
    if topic_id != -1:  # Ignore the -1 topic (outliers)
        print(f"Topic {topic_id}:")
        print(bert_topic.get_topic(topic_id))


Topic 0:
[('million', 0.023715019868909008), ('quarter', 0.022174213579705605), ('year', 0.01913483667309698), ('revenue', 0.019119997295380005), ('zacks', 0.01658252943215118), ('company', 0.0162650353896624), ('cent', 0.012967330641718747), ('rank', 0.012686737322500773), ('consensus', 0.012614959118743933), ('earnings', 0.01216444018030526)]
Topic 1:
[('amd', 0.06648051252337077), ('graphic', 0.02490345318267687), ('radeon', 0.023750654042430104), ('processor', 0.023070440742256924), ('gpu', 0.019340631148884183), ('ryzen', 0.018837502812469327), ('epyc', 0.016446617010257715), ('year', 0.015575105976734975), ('revenue', 0.014152314544202337), ('card', 0.013617960364400364)]
Topic 2:
[('nvidia', 0.030526409824141024), ('nvda', 0.015470702138198053), ('gaming', 0.014493230547864003), ('share', 0.01234473295020449), ('year', 0.011765787494306392), ('stock', 0.011522236193219997), ('company', 0.011344438787667511), ('analyst', 0.01104560897675967), ('chip', 0.010923551781912413), ('est

Here are possible labels for each topic based on the terms provided:

1. **Quarterly Earnings and Revenue**
2. **AMD Graphics and Processors**
3. **Nvidia Stock and Analyst Estimates**
4. **Nvidia Stock Trading Volume and Price Movements**
5. **Self-Driving Vehicles and Autonomous Technology**
6. **Stock Market Performance and Trends**
7. **Analyst Target Prices and Ratings**
8. **Tech Funds and ETFs**
9. **Intel and AI Technologies**
10. **Analyst Reports and Research**
11. **Market Indices and Currencies**
12. **Stock Price Resistance and Trends**
13. **Apple Financials and Market Position**
14. **Stock Market Indices and Movements**
15. **Nvidia Gaming and Revenue**
16. **Autonomous Driving Technology**
17. **U.S.-China Trade and Market Influence**
18. **AI, Nvidia, and Cloud Computing**
19. **Semiconductor Industry and Earnings**
20. **Earnings Growth and Investment Analysis**
21. **Oil Prices and Market Trends**
22. **Stock Market Trading and Investing**
23. **China-U.S. Trade Tensions**
24. **Gaming and Esports Market**
25. **Nvidia Stock Trading Details**
26. **Micron Memory and Semiconductor Market**
27. **Nvidia Stock and Trading Volume**
28. **Intel and Processor Industry**
29. **Social Media Advertising (Facebook, Snapchat, etc.)**
30. **Oil and Financial Market Indicators**
31. **Stock Market Trading Metrics**
32. **Cryptocurrency and Blockchain**
33. **Financial Futures and Trade Movements**
34. **Corporate Stocks and Financial Metrics**
35. **Qualcomm and Antitrust Issues**
36. **GAAP and Gaming Revenue**
37. **Semiconductor Industry Analysis**
38. **Investment Reports and Earnings**
39. **Steel Industry and Trade Tariffs**
40. **Trader Strategies and Market Analysis**
41. **Nvidia Stock Highs and Lows**
42. **Stock Market Performance**
43. **Nvidia RTX and Ray Tracing**
44. **Qualcomm and Chip Market**
45. **China Semiconductor Industry**
46. **SoftBank Vision Fund Investments**
47. **Mellanox and Data Networking Solutions**
48. **Stock Market Podcast and Analysis**
49. **Digital Storage Industry**
50. **Nvidia PEG Ratio and Value**
51. **Market Predictions and Investor Outlook**
52. **Storage Solutions and Flash Technology**
53. **Market Trends and Federal Policy**
54. **Shanghai Market and Stock Resistance Levels**
55. **Baidu Autonomous Driving Platform**
56. **Coronavirus Pandemic and Health**
57. **Earnings Reports and Analyst Forecasts**
58. **Blockchain ETFs and Investment Funds**

These labels summarize the primary themes or focus areas identified in each topic.

# For 2nd Iteration 

For this iteration: I did pre-processing steps. 

In [145]:
import pandas as pd
import numpy as np
import random
import torch
import re
import nltk
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from hdbscan import HDBSCAN
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Ensure nltk components are downloaded
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Initialize stop words and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Load data
df = pd.read_csv("/Users/vishalsehgal/Desktop/NLP/Assignment-1/Dataset/assignment-2-data.csv")

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)
torch.manual_seed(42)

# Define preprocessing function
def preprocess_text(doc):
    doc = re.sub(r'\W', ' ', str(doc))
    doc = doc.lower()
    doc = re.sub(r'\s+[a-z]\s+', ' ', doc)
    tokens = nltk.word_tokenize(doc)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return ' '.join(tokens)

# Apply preprocessing to Nvidia-related articles
nvidia_articles = [
    preprocess_text(doc) for doc in df[df['clean_content'].str.contains("nvidia", case=False, na=False)]['clean_content'].tolist()
]

# UMAP configuration for stable embeddings with random_state set, disabling parallelization for more determinism
umap_model = UMAP(
    random_state=42,
    n_neighbors=30,
    min_dist=0.1,
    metric="cosine",
    n_components=2,      # Default to 2 for consistency
    n_jobs=1             # Disabling parallel processing
)

# Adjusted HDBSCAN parameters with random_state equivalent, also set core_dist_n_jobs=1 for single-threaded execution
hdbscan_model = HDBSCAN(
    min_cluster_size=10,
    min_samples=5,
    prediction_data=True,
    approx_min_span_tree=False,
    core_dist_n_jobs=1     # Disabling parallel processing
)

# Set up the CountVectorizer with limited vocabulary and random state
vectorizer_model = CountVectorizer(lowercase=True, max_features=5000)

# Initialize BERTopic with controlled nr_topics
bert_topic = BERTopic(
    vectorizer_model=vectorizer_model,
    nr_topics=60,
    min_topic_size=15,
    calculate_probabilities=True,
    embedding_model=umap_model,
    hdbscan_model=hdbscan_model
)

# Train the model on the data and transform it into topics
topic_assigned_to_doc, probabilities = bert_topic.fit_transform(nvidia_articles)

# Check the actual number of topics generated
available_topics = bert_topic.get_topic_freq().shape[0] - 1  # exclude outlier topic (-1)
target_nr_topics = min(60, available_topics)

# Retrieve and display topics
topic_matrix = bert_topic.get_topic_info()
for topic_id in topic_matrix['Topic']:
    if topic_id != -1:  # Ignore the -1 topic (outliers)
        print(f"Topic {topic_id}:")
        print(bert_topic.get_topic(topic_id))


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vishalsehgal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vishalsehgal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vishalsehgal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Topic 0:
[('million', 0.031058667040019382), ('quarter', 0.027572435065493305), ('revenue', 0.023649734394869417), ('year', 0.022956133997230753), ('zacks', 0.019597658580272532), ('company', 0.017214624730812214), ('cent', 0.01597506352611448), ('rank', 0.015327899588276387), ('consensus', 0.015284978760477557), ('cash', 0.014518051003081423)]
Topic 1:
[('traded', 0.30743936290460316), ('seven', 0.13968310229977787), ('gmt', 0.13905561033118166), ('lowest', 0.13608521504930068), ('highest', 0.12521987649524965), ('day', 0.1251872216446241), ('nvidia', 0.12267190000850696), ('exchange', 0.11661544523984069), ('volume', 0.1134734716813733), ('session', 0.10768239108654434)]
Topic 2:
[('amd', 0.06575784154244459), ('graphic', 0.02492026407697873), ('radeon', 0.02312456721845143), ('processor', 0.022994345058067793), ('gpu', 0.02091474977210191), ('ryzen', 0.018364193304871854), ('epyc', 0.01604165700978441), ('year', 0.015383682311727672), ('revenue', 0.014322424586816627), ('card', 0.01

Here are suggested labels for each topic based on the given terms:

1. **Quarterly Financials and Revenue**
2. **Nvidia Stock Trading Volume and Movements**
3. **AMD Processors and Graphics Cards**
4. **Stock Resistance and Market Trends**
5. **U.S. Market Indices and Economic Events**
6. **U.S.-China Trade Relations and Market Impact**
7. **Nvidia Stock and Investor Sentiments**
8. **Analyst Ratings and Price Targets**
9. **Top Market Performers and Closures**
10. **Investment Podcasts and Market Analysis**
11. **Apple Products and Revenue**
12. **Semiconductor ETFs and Earnings**
13. **AI, Nvidia, and Cloud Computing**
14. **Nvidia Financials and Gaming Revenue**
15. **Micron and Memory Chip Market**
16. **Currency and Market Trends**
17. **Self-Driving Vehicles and Autonomous Technology**
18. **China-U.S. Trade and Technology**
19. **Intel Processors and Data Centers**
20. **Autonomous Vehicles and Market Players**
21. **Gaming and Esports Industry**
22. **Walmart and Retailer Performance**
23. **Trader Insights and Portfolio Management**
24. **Technology ETFs and Stock Sectors**
25. **Oil Prices and Energy Market**
26. **Semiconductor Industry and China Relations**
27. **Intel and AI Technology Developments**
28. **Oil Market Scenarios and Financial Metrics**
29. **Nvidia RTX and Ray Tracing Graphics**
30. **Market Predictions and Fed Impact**
31. **Stock Market Indicators and Trends**
32. **Social Media Advertising and Platforms**
33. **Corporate Stocks and Trading Movements**
34. **Earnings Reports and Financial Highlights**
35. **Cryptocurrency and Blockchain Technology**
36. **Intel Processors and PC Market**
37. **Qualcomm and Antitrust Issues**
38. **Big Tech Companies and Smart Devices**
39. **Autonomous Driving and Mobileye**
40. **Market Indices and Highs**
41. **Stock Growth and Earnings Projections**
42. **Stock Market Pre-Market Activity**
43. **EPS and Revenue Analysis**
44. **Qualcomm and Chip Industry**
45. **Samsung and Semiconductor Industry**
46. **Fed Rates and Economic Performance**
47. **AI, Machine Learning, and Robotics**
48. **Storage Solutions and Cloud Technology**
49. **Electronic Design and Verification (Cadence)**
50. **Nasdaq and Tech Stocks**
51. **Baidu Autonomous Driving Platform**
52. **Mellanox and Data Networking Solutions**
53. **Quarterly Earnings and Index Analysis**
54. **AI and Robotics ETFs**
55. **Digital Storage Industry (Western Digital)**
56. **Etsy and Quarterly Earnings**
57. **Blockchain ETFs and Digital Technology**
58. **Coronavirus Pandemic and Health Impacts**

These labels aim to encapsulate each topic's focus based on the prominent keywords.

# 3rd Iteration

added custom stop words used tfidf to imrpove word weighting 

In [146]:
import pandas as pd
import numpy as np
import random
import torch
import re
import nltk
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from hdbscan import HDBSCAN
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Ensure nltk components are downloaded
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Initialize stop words and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Define custom stop words
custom_stop_words = [
    "Quarter", "Week", "Today", "Period", "Session", "Billion", "Million", "Percent", "Index",
    "Investment", "Earnings", "Revenue", "Growth", "Rate", "High", "Volume", "Stock", "Target",
    "Price", "Analyst", "Market", "Fund", "Asset", "Allocation", "Solution", "Client", "Report",
    "Forecast", "Record", "Technology", "Industry", "System", "Platform", "Device", "Company",
    "Digital", "Sector"
] + list(stop_words)

# Load data
df = pd.read_csv("/Users/vishalsehgal/Desktop/NLP/Assignment-1/Dataset/assignment-2-data.csv")

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)
torch.manual_seed(42)

# Define preprocessing function
def preprocess_text(doc):
    doc = re.sub(r'\W', ' ', str(doc))
    doc = doc.lower()
    doc = re.sub(r'\s+[a-z]\s+', ' ', doc)
    tokens = nltk.word_tokenize(doc)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in custom_stop_words]
    return ' '.join(tokens)

# Apply preprocessing to Nvidia-related articles
nvidia_articles = [
    preprocess_text(doc) for doc in df[df['clean_content'].str.contains("nvidia", case=False, na=False)]['clean_content'].tolist()
]

# UMAP configuration for stable embeddings with random_state set, disabling parallelization for more determinism
umap_model = UMAP(
    random_state=42,
    n_neighbors=30,
    min_dist=0.1,
    metric="cosine",
    n_components=2,      # Default to 2 for consistency
    n_jobs=1             # Disabling parallel processing
)

# Adjusted HDBSCAN parameters with random_state equivalent, also set core_dist_n_jobs=1 for single-threaded execution
hdbscan_model = HDBSCAN(
    min_cluster_size=10,
    min_samples=5,
    prediction_data=True,
    approx_min_span_tree=False,
    core_dist_n_jobs=1     # Disabling parallel processing
)

# Set up the CountVectorizer with limited vocabulary and random state
vectorizer_model = CountVectorizer(lowercase=True, max_features=5000)

# Initialize BERTopic with controlled nr_topics
bert_topic = BERTopic(
    vectorizer_model=vectorizer_model,
    nr_topics=60,
    min_topic_size=15,
    calculate_probabilities=True,
    embedding_model=umap_model,
    hdbscan_model=hdbscan_model
)

# Train the model on the data and transform it into topics
topic_assigned_to_doc, probabilities = bert_topic.fit_transform(nvidia_articles)

# Check the actual number of topics generated
available_topics = bert_topic.get_topic_freq().shape[0] - 1  # exclude outlier topic (-1)
target_nr_topics = min(60, available_topics)

# Retrieve and display topics
topic_matrix = bert_topic.get_topic_info()
for topic_id in topic_matrix['Topic']:
    if topic_id != -1:  # Ignore the -1 topic (outliers)
        print(f"Topic {topic_id}:")
        print(bert_topic.get_topic(topic_id))


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vishalsehgal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vishalsehgal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vishalsehgal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Topic 0:
[('traded', 0.307497169230812), ('seven', 0.1397093662332609), ('gmt', 0.13908175628040567), ('lowest', 0.13611080248956536), ('highest', 0.12524342097881908), ('day', 0.12479199490468819), ('nvidia', 0.12269681589961898), ('exchange', 0.11663737187410388), ('volume', 0.11350496458946052), ('session', 0.10770263808213815)]
Topic 1:
[('nvidia', 0.028200642314781126), ('gaming', 0.01959077902629935), ('quarter', 0.01660059360833836), ('year', 0.016418446943391157), ('revenue', 0.015651927594537658), ('billion', 0.014735907972854538), ('nvda', 0.013445395543387586), ('fiscal', 0.0131628276287997), ('gpu', 0.012673526803970025), ('company', 0.01243996142341068)]
Topic 2:
[('nyse', 0.01818255441954104), ('percent', 0.016963386621111003), ('dow', 0.01641136464152205), ('point', 0.015123934912157834), ('trump', 0.014648664341060401), ('nasdaq', 0.013379086864634696), ('fell', 0.012741021337378545), ('rose', 0.012516387763277516), ('index', 0.012352390529544458), ('rate', 0.0113943666

Here are suggested labels for each topic:

1. **Nvidia Stock Trading Volume and Session Activity**
2. **Nvidia Financials and Gaming Revenue**
3. **U.S. Market Indices and Economic Indicators**
4. **Analyst Reports and Investment Research**
5. **Autonomous Vehicles and Driving Technology**
6. **Market Performers and Price Movements**
7. **Nvidia Stock Price Targets and Analyst Ratings**
8. **Quarterly Financial Metrics and Cash Flow**
9. **Stock Price Resistance and Market Trends**
10. **Tech Sector and ETF Performance**
11. **Apple Products and Revenue Trends**
12. **Investment Podcasts and Stock Market Analysis**
13. **AI and Nvidia's Role in Technology**
14. **Semiconductor ETF and Industry Performance**
15. **Market Currencies and Trade**
16. **China-U.S. Trade Tensions and Tech Exposure**
17. **Micron Memory and Semiconductor Demand**
18. **China-U.S. Trade and Financial Market Impact**
19. **Autonomous Vehicles and Automotive Technology**
20. **AMD Processors and Revenue Trends**
21. **Intel Processors and Data Centers**
22. **Earnings Growth and Financial Estimates**
23. **Gaming and Esports Market**
24. **Walmart and Retail Performance**
25. **AMD Radeon Graphics Cards**
26. **Oil Prices and Energy Market Trends**
27. **Trader Strategies and Portfolio Management**
28. **Intel Processors and PC Market**
29. **Oil Market Scenarios and Financial Targets**
30. **Semiconductor Industry and China Relations**
31. **Cryptocurrency and Blockchain Mining**
32. **Stock Market Indicators and Support Levels**
33. **Social Media Advertising and Platforms**
34. **Corporate Stocks and Financial Metrics**
35. **Nvidia RTX and Ray Tracing Technology**
36. **Earnings Surprise and Analyst Predictions**
37. **Qualcomm and EU Antitrust Issues**
38. **Market Predictions and Fed Influence**
39. **Stock Market Premarket Activity**
40. **Nasdaq and Midday Tech Performance**
41. **Samsung and Semiconductor Market**
42. **Data Storage Solutions and Cloud**
43. **Qualcomm and 5G Technology**
44. **Travel Industry Earnings and Reviews**
45. **AI, Machine Learning, and Robotics**
46. **NetApp Cloud Solutions and Earnings**
47. **Smart Devices and Digital Assistants**
48. **Intel AI and Machine Learning Initiatives**
49. **Mellanox and Data Networking Technology**
50. **Baidu's Autonomous Driving Platform**
51. **Earnings and Revenue Forecasts**
52. **Western Digital Storage Solutions**
53. **Stock Resistance and Trading Analysis**
54. **FANG Stocks and Market Bubbles**
55. **Electronic Design and Verification Tools**
56. **AMD in Cryptocurrency Mining**
57. **Blockchain ETFs and Digital Assets**
58. **Coronavirus Pandemic and Health Impact**

These labels aim to reflect the primary themes of each topic based on prominent terms.

In [147]:
from sklearn.metrics import silhouette_score

# Get document embeddings
embeddings = bert_topic.transform(nvidia_articles)[1]

# Reshape embeddings to 2D
embeddings = embeddings.reshape(-1, 1)

# Assess initial topic quality
silhouette_avg = silhouette_score(embeddings, topic_assigned_to_doc)
print(f"Silhouette Score: {silhouette_avg}")


ValueError: Found input variables with inconsistent numbers of samples: [179537, 3043]

In [ ]:
# Visualize the frequency of topics
bert_topic_model.visualize_barchart()

NameError: name 'bert_topic_model' is not defined

In [ ]:
# Visualize topics using a hierarchical structure
bert_topic_model.visualize_hierarchy()

In [ ]:
# Visualize the similarity between topics
bert_topic_model.visualize_heatmap()

2nd iteration

In [ ]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

# Define a custom vectorizer to include bigrams and trigrams
vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=list(stop_words))

# Adjust min_topic_size
bert_topic_model = BERTopic(
    vectorizer_model=vectorizer_model,
    min_topic_size=10,
    n_gram_range=(1, 3)
)

# Fit the model
topics, probabilities = bert_topic_model.fit_transform(nvidia_articles)

KeyboardInterrupt: 

In [ ]:
topic_info = bert_topic_model.get_topic_info()
print(topic_info.head())


   Topic  Count                              Name  \
0     -1    987      -1_year_stock_company_nasdaq   
1      0    146    0_million_quarter_year_revenue   
2      1    118      1_amd_graphic_processor_year   
3      2     75  2_resistance_stock_around_nasdaq   
4      3     72        3_vehicle_driving_car_self   

                                      Representation  \
0  [year, stock, company, nasdaq, market, earning...   
1  [million, quarter, year, revenue, zacks, compa...   
2  [amd, graphic, processor, year, radeon, gpu, r...   
3  [resistance, stock, around, nasdaq, towards, b...   
4  [vehicle, driving, car, self, self driving, au...   

                                 Representative_Docs  
0  [immediate release chicago il may highlight im...  
1  [keeping positive surprise history alive proof...  
2  [advanced micro device nasdaq amd q4 earnings ...  
3  [trading future point year yield oil dollar in...  
4  [intel corporation nasdaq intc recent deal win...  


In [ ]:
for topic_id in topic_info['Topic']:
    if topic_id != -1:
        print(f"Topic {topic_id}:")
        print(bert_topic_model.get_topic(topic_id))

Topic 0:
[('million', 0.016118678565553532), ('quarter', 0.015267606659485816), ('year', 0.014334045194907951), ('revenue', 0.01265395647732489), ('zacks', 0.011589394274718487), ('company', 0.010713142503972421), ('year year', 0.009761059323096108), ('earnings', 0.007984736034069248), ('cent', 0.007981690185298583), ('zacks rank', 0.007837529657295747)]
Topic 1:
[('amd', 0.0310315616874522), ('graphic', 0.011025772915340072), ('processor', 0.009758602289134229), ('year', 0.009525636869971505), ('radeon', 0.009270725125074063), ('gpu', 0.00903017999015608), ('revenue', 0.007706715571472205), ('ryzen', 0.007243251169554153), ('quarter', 0.007079640080775927), ('market', 0.006516863411960394)]
Topic 2:
[('resistance', 0.018169579190449234), ('stock', 0.015067911900740872), ('around', 0.013748372805314185), ('nasdaq', 0.012581939316324278), ('towards', 0.010759906947704956), ('break', 0.010438912239212493), ('higher', 0.010148418855056612), ('roku', 0.00999659750851234), ('back', 0.009216

Here are suggested labels for each topic:

- **Topic 0**: AMD and Graphics Processors
- **Topic 1**: Stock Resistance and Price Movement
- **Topic 2**: Autonomous Vehicles and Tesla
- **Topic 3**: Technology Funds and ETFs
- **Topic 4**: Stock Analysis and Research Reports
- **Topic 5**: Stock Market Performance (NASDAQ and NYSE)
- **Topic 6**: Intel and AI Technology
- **Topic 7**: Apple Product Updates and Financials
- **Topic 8**: Investment Advice and Podcasts
- **Topic 9**: Nvidia Price Targets and Stock Ratings
- **Topic 10**: Self-Driving Cars and Automotive Industry
- **Topic 11**: Cryptocurrency and Blockchain
- **Topic 12**: Nvidia, AI, and Autonomous Vehicles
- **Topic 13**: Market Yields and Currency Movements
- **Topic 14**: Nvidia Revenue and Gaming Industry
- **Topic 15**: Stock Market Indexes and Performers
- **Topic 16**: Nvidia Stock and Investor Interest
- **Topic 17**: Financial Reports and Cash Flow
- **Topic 18**: Gaming and Esports Industry
- **Topic 19**: China and Trade Policies
- **Topic 20**: Micron and Memory Chip Industry
- **Topic 21**: Trade Relations and Market Reactions
- **Topic 22**: Nvidia Stock Trading Trends
- **Topic 23**: Intel and Processor Market
- **Topic 24**: Stock Market Strategies and Trends
- **Topic 25**: Oil Prices and Energy Sector
- **Topic 26**: Stock Futures and Market Indicators
- **Topic 27**: Oil Market Scenarios and Targets
- **Topic 28**: Retail and Earnings Reports
- **Topic 29**: Stock Market Indicators and VIX
- **Topic 30**: Corporate Stocks and Trading Activity
- **Topic 31**: Social Media Advertising and Platforms
- **Topic 32**: Semiconductor Stocks and Earnings
- **Topic 33**: Nvidia Stock Trading Volume
- **Topic 34**: Qualcomm and Antitrust Issues
- **Topic 35**: Chinese Semiconductor Market
- **Topic 36**: Nasdaq Trading Patterns (Friday)
- **Topic 37**: Nvidia Revenue and Quarterly Earnings
- **Topic 38**: Nvidia Ray Tracing and Gaming
- **Topic 39**: Company Earnings Reports
- **Topic 40**: Financial Metrics and Cash Flow
- **Topic 41**: Zacks Rank and Quarterly Earnings
- **Topic 42**: Nasdaq Trading Patterns (Monday)
- **Topic 43**: Quarterly Financial Metrics and GAAP
- **Topic 44**: Nasdaq Trading Patterns (Wednesday)
- **Topic 45**: Stock Market Highs and Percent Changes
- **Topic 46**: Earnings Growth and Research Analysis
- **Topic 47**: Market Insights and Predictions
- **Topic 48**: EPS, Revenue, and Analyst Expectations
- **Topic 49**: Stock Focus and Premarket Activity
- **Topic 50**: Nasdaq Trading Patterns (Tuesday)
- **Topic 51**: Qualcomm and Semiconductor Technology
- **Topic 52**: Pure Storage and Cloud Solutions
- **Topic 53**: Cadence Design Systems and Verification
- **Topic 54**: Mellanox and Networking Solutions
- **Topic 55**: Federal Reserve and Economic Indicators
- **Topic 56**: Baidu and Autonomous Driving Platform
- **Topic 57**: Nvidia Stock Cycle Analysis
- **Topic 58**: NetApp and Consensus Estimates
- **Topic 59**: Nasdaq and Technology Stocks
- **Topic 60**: Quarterly Earnings and Revenue Growth
- **Topic 61**: Nasdaq Trading Patterns (Friday)
- **Topic 62**: Western Digital and Storage Solutions
- **Topic 63**: TSMC, Samsung, and Chip Demand
- **Topic 64**: Nasdaq Trading Patterns (Thursday)
- **Topic 65**: Semiconductor Industry and 5G Expansion

3rd iteration: dimensionality reductioon 

In [ ]:
from umap import UMAP
from hdbscan import HDBSCAN

# Adjust UMAP parameters
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

# Adjust HDBSCAN parameters
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom')

# Initialize BERTopic with custom models
bert_topic_model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model
)

# Fit the model
topics, probabilities = bert_topic_model.fit_transform(nvidia_articles)

KeyboardInterrupt: 

In [ ]:
topic_info = bert_topic_model.get_topic_info()
print(topic_info)


    Topic  Count                                               Name  \
0      -1    862                       -1_stock_company_year_nasdaq   
1       0    315                          0_nyse_nasdaq_market_week   
2       1    294                     1_quarter_year_million_revenue   
3       2    124                       2_amd_graphic_year_processor   
4       3     77                         3_intel_ai_chip_technology   
5       4     74                   4_resistance_stock_around_nasdaq   
6       5     71                         5_driving_vehicle_car_self   
7       6     69                 6_nvidia_price target_target_price   
8       7     62                            7_inc_close_nasdaq_nyse   
9       8     59                           8_nvidia_nvda_stock_year   
10      9     56                      9_quarter_year_nvidia_revenue   
11     10     55                      10_apple_iphone_company_zacks   
12     11     54            11_analyst_zacks analyst_zacks_research   
13    

In [ ]:

for topic_id in topic_info['Topic']:
    if topic_id != -1:
        print(f"Topic {topic_id}:")
        print(bert_topic_model.get_topic(topic_id))

Topic 0:
[('nyse', 0.008547667603765473), ('nasdaq', 0.007863873928492271), ('market', 0.0074885610178155124), ('week', 0.00722716526385484), ('trade', 0.0067932142688341005), ('index', 0.006351911009927603), ('point', 0.005887586434053044), ('stock', 0.005840394544291641), ('china', 0.005830784131462191), ('oil', 0.005557435845929734)]
Topic 1:
[('quarter', 0.014181895594029967), ('year', 0.014043888701808055), ('million', 0.013971375645889732), ('revenue', 0.011939211148347104), ('company', 0.01181984981728489), ('zacks', 0.011396922097022344), ('year year', 0.008469373658685225), ('earnings', 0.00801302245816429), ('zacks rank', 0.0076508577948204365), ('rank', 0.007501657408345907)]
Topic 2:
[('amd', 0.03377381100942366), ('graphic', 0.011768054411315794), ('year', 0.010990356898298284), ('processor', 0.01050045802460603), ('radeon', 0.00970755177180716), ('gpu', 0.009630549084600053), ('revenue', 0.008638277048711313), ('quarter', 0.007999670614550449), ('ryzen', 0.007524497261751

Here are suggested topic labels based on the keywords provided for each topic:

- **Topic 0**: Stock Market and Trade Relations
- **Topic 1**: Financial Metrics and Earnings
- **Topic 2**: AMD and Graphics Processors
- **Topic 3**: Stock Resistance and Price Movement
- **Topic 4**: Intel and AI Technology
- **Topic 5**: Nvidia Stock Ratings and Price Targets
- **Topic 6**: Autonomous Vehicles and Self-Driving Technology
- **Topic 7**: Stock Market Performers (NASDAQ and NYSE)
- **Topic 8**: Zacks Analyst Reports
- **Topic 9**: Semiconductor Industry and ETFs
- **Topic 10**: Nvidia Stock and Investor Interest
- **Topic 11**: Technology ETFs and Growth Stocks
- **Topic 12**: Apple Product and Financial Updates
- **Topic 13**: Investment Podcasts and Market Insights
- **Topic 14**: Nasdaq Stock Trends and Investing
- **Topic 15**: Bitcoin and Blockchain
- **Topic 16**: Autonomous Driving Technology (Nvidia Focus)
- **Topic 17**: Earnings Growth and Financial Estimates
- **Topic 18**: Gaming and Esports Industry
- **Topic 19**: Nvidia Stock Trading Volume (Positive Movement)
- **Topic 20**: Nvidia Stock Trading Volume (Negative Movement)
- **Topic 21**: Nvidia Revenue and Gaming Segment
- **Topic 22**: Micron and Memory Industry
- **Topic 23**: Stock Trading and Portfolio Management
- **Topic 24**: Storage Industry and Pure Storage
- **Topic 25**: Intel and Chip Market
- **Topic 26**: Nvidia and AI Applications
- **Topic 27**: Oil Market Scenarios and Target Prices
- **Topic 28**: Social Media Platforms and Advertising
- **Topic 29**: Stock Trading Indicators and VIX
- **Topic 30**: Corporate Stocks and Trading Patterns
- **Topic 31**: Nvidia Stock Trading Volume (Negative Movement)
- **Topic 32**: Nvidia Gaming and Ray Tracing Technology
- **Topic 33**: Qualcomm and Antitrust Issues
- **Topic 34**: Nvidia Stock Trading Volume (Friday Patterns)
- **Topic 35**: Semiconductor Industry and Trade Relations
- **Topic 36**: Nvidia Stock Trading Volume (Monday Patterns)
- **Topic 37**: Stock Market Indices and Percentage Changes
- **Topic 38**: Cadence Design and IP Verification
- **Topic 39**: Financial Metrics and Quarterly Earnings
- **Topic 40**: Company Earnings Reports and Missed Expectations
- **Topic 41**: EPS and Revenue Estimates
- **Topic 42**: Nvidia Stock Trading Volume (Tuesday Patterns)
- **Topic 43**: Qualcomm, Intel, and Chip Technology
- **Topic 44**: Retail Industry and Earnings